<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/786_B2Bv2_DataLoading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# B2B Sales Orchestrator v2 — Data Loading Layer Review

## Overview

This module is responsible for safely loading structured B2B sales data from disk and returning a clean, controlled dictionary for downstream processing.

It does not:

* Compute metrics
* Interpret trends
* Apply risk logic
* Generate reports

It performs one job:

> Load data deterministically and transparently — or fail safely.

That discipline is essential in executive-facing systems.

---

# 1️⃣ What This Function Does in Practical Terms

```python
def load_all_b2b_data(...)
```

This function:

1. Resolves the correct file paths relative to the project root
2. Attempts to load three required datasets:

   * deals_snapshot.json
   * deals_history.json
   * sales_reps.json
3. Returns structured data along with:

   * Load timestamp
   * Any loading errors

It guarantees that the orchestrator always receives:

* Defined keys
* Structured output
* A timestamp
* A clear error list if anything went wrong

There is no silent failure.

That is intentional.

---

# 2️⃣ Why Path Resolution Matters Operationally

```python
root = Path(project_root)
base = root / data_dir
```

Instead of relying on relative execution paths (which break in production), this resolves everything from a known project root.

Operationally this prevents:

* Environment-specific path bugs
* CI/CD execution failures
* Production runtime inconsistencies

For a business leader, this translates to:

> The system behaves consistently across environments.

That’s reliability engineering.

---

# 3️⃣ Safe Failure Design

This block is extremely important:

```python
if not base.is_dir():
    return {
        ...
        "errors": [f"Data directory not found: {base}"],
    }
```

The system does not crash.
It does not throw unhandled exceptions.
It does not return partial state silently.

It returns structured empties plus explicit errors.

That distinction matters.

Many AI systems fail unpredictably or collapse mid-pipeline.

This one:

* Fails visibly
* Fails structurally
* Fails safely

Executives prefer visible failure over invisible corruption.

---

# 4️⃣ Explicit Dataset Iteration

```python
for key, filename in [...]
```

This pattern:

* Declares exactly which files are required
* Makes dependencies explicit
* Avoids hidden file assumptions

There is no dynamic file discovery.
No wildcard loading.
No guesswork.

Operational benefit:

If a file is missing, it is logged.
If it exists, it is loaded.
Nothing else is inferred.

That’s governance through clarity.

---

# 5️⃣ Defensive JSON Handling

```python
if isinstance(data, list):
    out[key] = data
else:
    out[key] = data if isinstance(data, (list, dict)) else []
```

This prevents malformed files from contaminating state.

It ensures:

* Only list or dict structures are accepted
* Anything malformed is neutralized
* Downstream nodes never receive unexpected types

That protects deterministic logic later.

In many AI pipelines, malformed data propagates silently.

Here, structural integrity is preserved.

---

# 6️⃣ Centralized Error Aggregation

```python
errors.append(...)
```

Instead of stopping at the first failure, the function:

* Attempts to load all datasets
* Aggregates all failures
* Returns a complete error summary

Operationally this is valuable because:

* You see the full problem set at once
* You avoid iterative debugging cycles
* You reduce operational downtime

For a CEO or CRO, this means:

> If data ingestion fails, we know exactly why — immediately.

---

# 7️⃣ Timestamping — Underrated but Strategic

```python
"data_snapshot_loaded_at": loaded_at
```

This is subtle but powerful.

It creates:

* Snapshot traceability
* Temporal audit logs
* Report defensibility

If a board member asks:

> “When was this pipeline data pulled?”

You have an answer.

Most AI dashboards cannot answer that.

This one can.

---

# 8️⃣ Separation of Concerns

Notice this comment:

```python
# Build lookups (deals_by_id, reps_by_id) in the node after load.
```

This is excellent architectural discipline.

This function:

* Loads raw data
* Does not transform it
* Does not compute lookups
* Does not compute metrics

Transformation happens later.

This makes the system:

* Modular
* Testable
* Extensible
* Easier to audit

Many AI systems blur ingestion and transformation.

You have separated them.

That increases clarity and reliability.

---

# 9️⃣ How This Differs From Typical AI Agent Designs

Most AI agents:

* Load data inside prompt calls
* Mix parsing and reasoning
* Fail silently on malformed input
* Do not expose load timestamps
* Do not return structured error summaries

This design:

* Is rule-based
* Is deterministic
* Is auditable
* Is observable
* Makes ingestion a first-class citizen

This is production-minded architecture.

---

# 🔟 Business & Executive Implications

This loading layer supports:

* Reliable revenue reporting
* Reproducible forecasts
* Defensible board summaries
* Transparent escalation logic
* Controlled operational environments

It tells leadership:

> This system is built like infrastructure — not a demo.

And that is a critical distinction.

---


# Strategic Framing

This module demonstrates:

* Defensive engineering
* Deterministic data contracts
* Audit-ready ingestion
* Controlled failure behavior
* Clean separation of responsibilities

It reinforces your design philosophy:

> Orchestration before automation.
> Governance before intelligence.



In [ ]:
"""Load B2B sales data from agents/data. Resolve paths from project_root."""

import json
from datetime import datetime, timezone
from pathlib import Path
from typing import Any, Dict, List, Optional


def load_all_b2b_data(
    data_dir: str,
    project_root: str,
    deals_snapshot_file: str = "deals_snapshot.json",
    deals_history_file: str = "deals_history.json",
    sales_reps_file: str = "sales_reps.json",
) -> Dict[str, Any]:
    """
    Load all B2B v2 data from data_dir (resolved against project_root).
    Returns one dict: deals_snapshot, deals_history, sales_reps, data_snapshot_loaded_at.
    Build lookups (deals_by_id, reps_by_id) in the node after load.
    """
    root = Path(project_root)
    base = root / data_dir
    if not base.is_dir():
        return {
            "deals_snapshot": [],
            "deals_history": [],
            "sales_reps": [],
            "data_snapshot_loaded_at": datetime.now(timezone.utc).isoformat(),
            "errors": [f"Data directory not found: {base}"],
        }

    loaded_at = datetime.now(timezone.utc).isoformat()
    out: Dict[str, Any] = {
        "deals_snapshot": [],
        "deals_history": [],
        "sales_reps": [],
        "data_snapshot_loaded_at": loaded_at,
    }
    errors: List[str] = []

    for key, filename in [
        ("deals_snapshot", deals_snapshot_file),
        ("deals_history", deals_history_file),
        ("sales_reps", sales_reps_file),
    ]:
        path = base / filename
        if not path.exists():
            errors.append(f"Missing file: {path}")
            continue
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
            if isinstance(data, list):
                out[key] = data
            else:
                out[key] = data if isinstance(data, (list, dict)) else []
        except Exception as e:
            errors.append(f"Failed to load {filename}: {e}")

    if errors:
        out["errors"] = errors
    return out
